In [1]:
from collections import deque
import sys, time, resource
from heapq import heappush, heappop

In [2]:
MOVES = ("Up", "Down", "Left", "Right")#order in " UDLR "
MOVEBY = [-3, 3, -1, 1]
GOAL = (0, 1, 2, 3, 4, 5, 6, 7, 8)
ACTION = [(1, 3), (1, 2, 3), (1, 2),(0, 1, 3), (0, 1, 2, 3), (0, 1, 2),(0, 3), (0, 2, 3), (0, 2)]
M_Distance = [0,1,2,1,2,3,2,3,4]
class TreeNode(object):
    def __init__(self,state,parent,by,manhattan_distance):
        self.state = state #current positions 
        self.parent = parent#the former node
        self.by = by #the former action
        self.depth = parent.depth + 1 if parent else 0 #the current depth
        self.cost = parent.cost +1 if parent else 0 #the current cost
        self.zero = self.state.index(0) #index of zero
        self.action = ACTION[self.zero] #next valid actions
        self.reward = manhattan_distance+self.depth #in A*

In [3]:
def manhattan_distance(node):#A* Manhattan priority function of the node
    distance = 0
    curpos = -1
    for num in node.state: #num is the number as well as the goal position
        curpos+=1        #curpos is the current position
        if num != 0:
            distance += M_Distance[abs(curpos - num)]
    return distance      
def get_history(node):#get the path from head to current node
    path = []
    while node.parent:  
        path = [node.by] + path
        node = node.parent
    return path
def generate_next(node,A):#get the valid child nodes
    Next = []
    reward = manhattan_distance(node) if A else 0 
    for a in list(node.action):
        target = node.zero + MOVEBY[a] #the target to switch
        newstate = list(node.state) #newstate to return
        newstate[node.zero],newstate[target] = newstate[target],newstate[node.zero]#switch
        Next.append(TreeNode(tuple(newstate), node, a, reward))#new node with parent and action
    return Next
def display_board(state):
    if state != None:
        print ("-------------")
        print ("| %i | %i | %i |" % (state[0], state[1], state[2]))
        print ("-------------")
        print ("| %i | %i | %i |" % (state[3], state[4], state[5]))
        print ("-------------")
        print ("| %i | %i | %i |" % (state[6], state[7], state[8]))
        print ("-------------")
    else:
        print("Invalid state")
def bfs(state):    
    print '--------------BFS--------------'
    display_board(state)
    #things to return
    result = {'path_to_goal' : deque(), #return the path to goal node
    'cost_of_path' : 0, #return the goal node cost
    'search_depth' : 0, #return the goal node depth
    'nodes_expanded' : 0,#sum the total number of nodes visited
    'max_search_depth' : 0, #by max(every visited nodes' search depth)
    'running_time' : 0, #calculate the end_time - begin_time
    'max_ram_usage' : 0} #
    #things to return
    frontier = deque([TreeNode(state,None,None,0)])
    explored = set([state])
    timenow = time.time() #the begin_time
    while frontier:
        CurNode = frontier.popleft()#pop off results in UDLR order.
        if CurNode.state == GOAL:
            result['search_depth'] = CurNode.depth
            result['path_to_goal'] = get_history(CurNode)
            result['running_time'] = time.time()- timenow
            return result
        deeper = False
        for child in generate_next(CurNode, False):#Push onto the queue in UDLR order;
            if child.state not in explored:
                frontier.append(child)
                explored.add(child.state)
                deeper = True
        result['nodes_expanded']+=1
        result['max_search_depth']= max(result['max_search_depth'],CurNode.depth+1) if deeper==True else result['max_search_depth']
        result['max_ram_usage'] = max(result['max_ram_usage'], resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1000000.0)
    return None
def dfs(stat):   
    print '--------------DFS-----------------'
    display_board(state)
    #things to return
    result = {'path_to_goal' : deque(), #return the path to goal node
    'cost_of_path' : 0, #return the goal node cost
    'search_depth' : 0, #return the goal node depth
    'nodes_expanded' : 0,#sum the total number of nodes visited
    'max_search_depth' : 0, #by max(every visited nodes' search depth)
    'running_time' : 0, #calculate the end_time - begin_time
    'max_ram_usage' : 0} #
    #things to return
    frontier = deque([TreeNode(state,None,None,0)])
    explored = set([state])
    timenow = time.time() #the begin_time
    while frontier:
        CurNode = frontier.pop()#pop off results in UDLR order.
        if CurNode.state == GOAL:
            result['search_depth']= CurNode.depth
            result['path_to_goal'] = get_history(CurNode)
            result['running_time'] = time.time()- timenow
            return result
        deeper = False
        for child in reversed(generate_next(CurNode, False)):#Push onto the stack in reverse-UDLR order;
            if child.state not in explored:
                frontier.append(child)
                explored.add(child.state)
                deeper = True
        result['max_search_depth']= max(result['max_search_depth'],CurNode.depth+1) if deeper==True else result['max_search_depth']
        result['nodes_expanded'] += 1
        result['max_ram_usage'] = max(result['max_ram_usage'], resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1000000.0)
    return None



In [4]:
def ast(state):    
    print '--------------AST--------------'
    display_board(state)
    #things to return
    result = {'path_to_goal' : deque(), #return the path to goal node
    'cost_of_path' : 0, #return the goal node cost
    'search_depth' : 0, #return the goal node depth
    'nodes_expanded' : 0,#sum the total number of nodes visited
    'max_search_depth' : 0, #by max(every visited nodes' search depth)
    'running_time' : 0, #calculate the end_time - begin_time
    'max_ram_usage' : 0} #
    #things to return
    timenow = time.time() #the begin_time
    Root = TreeNode(state,None,None,0) #Root node
    Unit = [manhattan_distance(Root),Root]
    frontier = [Unit] #nodes to be explored
    explored = set([state]) #state explored
    minimum_distance_of_state = {Root.state:Unit}#remember the minimum distance
    while frontier:
        distance, CurNode = heappop(frontier) #pop off results in UDLR order.
        if CurNode == None:
            continue
        explored.add(CurNode.state)
        result['max_search_depth'] = max(result['max_search_depth'],CurNode.depth)
        if CurNode.state == GOAL:
            result['search_depth'] = CurNode.depth
            result['path_to_goal'] = get_history(CurNode)
            result['running_time'] = time.time()- timenow
            return result
        deeper = False
        for child in generate_next(CurNode,True):
            state = child.state
            if state not in explored:
                distance = manhattan_distance(child)+child.depth
                Unit = [distance,child]
                if state in minimum_distance_of_state:
                    old = minimum_distance_of_state[state]
                    if distance<old[0]:
                        heappush(frontier,Unit)
                        old[1]=None
                        minimum_distance_of_state[state] = Unit
                        deeper = True
                else:
                    heappush(frontier,Unit)
                    minimum_distance_of_state[state] = Unit
                    deeper = True
        result['max_search_depth']= max(result['max_search_depth'],CurNode.depth+1) if deeper else result['max_search_depth']
        result['nodes_expanded']+=1
        result['max_ram_usage'] = max(result['max_ram_usage'], resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1000000.0)
    return None

In [5]:
if __name__ == '__main__':
    testcases = [[3, 1, 2, 0, 4, 5, 6, 7, 8],
                 [1, 2, 5, 3, 4, 0, 6, 7, 8],
                 [6, 1, 8, 4, 0, 2, 7, 3, 5],
                 [8, 6, 4, 2, 1, 3, 5, 7, 0]]
    for t in testcases:
        result = ast(tuple(t))
        path = [MOVES[m] for m in result['path_to_goal']]
        print 'path_to_goal:', path
        print "cost_of_path:", len(path)
        print "nodes_expanded:", result['nodes_expanded']
        print "search_depth:", result['search_depth']
        print "max_search_depth:", result['max_search_depth']
        print "running_time: %.8f" %(result['running_time'])
        print "max_ram_usage: %.8f"  %(result['max_ram_usage'])

--------------AST--------------
-------------
| 3 | 1 | 2 |
-------------
| 0 | 4 | 5 |
-------------
| 6 | 7 | 8 |
-------------
path_to_goal: ['Up']
cost_of_path: 1
nodes_expanded: 1
search_depth: 1
max_search_depth: 1
running_time: 0.00014877
max_ram_usage: 40.65280000
--------------AST--------------
-------------
| 1 | 2 | 5 |
-------------
| 3 | 4 | 0 |
-------------
| 6 | 7 | 8 |
-------------
path_to_goal: ['Up', 'Left', 'Left']
cost_of_path: 3
nodes_expanded: 3
search_depth: 3
max_search_depth: 3
running_time: 0.00030684
max_ram_usage: 40.67737600
--------------AST--------------
-------------
| 6 | 1 | 8 |
-------------
| 4 | 0 | 2 |
-------------
| 7 | 3 | 5 |
-------------
path_to_goal: ['Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Right', 'Down', 'Left', 'Up', 'Left', 'Up', 'Right', 'Right', 'Down', 'Down', 'Left', 'Left', 'Up', 'Up']
cost_of_path: 20
nodes_expanded: 843
search_depth: 20
max_search_depth: 20
running_time: 0.04047513
max_ram_usage: 42.19289600
--------------